# Modes of operation:

Modalità di funzionamento dei cifrari a blocchi per estendere uno schema di encryption <b>fixed length</b> a messaggi di lunghezza arbitraria

## ECB-mode:

Eletronic CodeBook 

<div>
<img src="images/ecb_mode.png" width="800"/>
</div>

$X = x_1 \ ... \ x_t \quad$ plaintext, con $\ |x_i|=n \quad \forall i \in 1..t$\
$Y = y_1 \ ... \ y_t \quad$ ciphertext: $\ Y=E_k(X)$

È assolutamente <b>insicuro</b> e praticamente inutilizzabile per fini pratici.\
Non introduce mai della <b>randomness</b>: la funzione pseudo-random è un algoritmo <b>deterministico</b> (si chiama proprio funzione).\
$M$ inoltre conosce $1^n$ (lunghezza della chiave) per il principio di Kerchoffs.

##### Caratteristiche: 

- Richiede che $F_k$ sia invertibile per eseguire la decryption: $\quad x_1 = F_k^{-1}(y_1) \ \ ... \ \ x_t = F_k^{-1}(y_t)$ 
- L'encryption è totalmente <b>parallelizzabile</b>
- La decryption è <b>random access</b>: posso decifrare i blocchi in qualunque ordine e poi rimetterli insieme

Non è</b> neanche EAV-sicuro:

$M \rightarrow A: \quad X_0=x_0x_0 \quad X_1=x_0x_1 \quad (x_0 \neq x_1)$\
$A \rightarrow M: \quad y = E_k(X_b) = E_k(X[1]) \ E_k(X[2]) \quad b \leftarrow \{0,1\} \quad $ <i>l'encryption è la concatenazione delle due encryption (funzione deterministica) con la stessa chiave</i>\
$M: \quad b_m \ = \ (Y[1] == Y[2]) \ \ ? \ \ 0 \ \ : \ \ 1 \quad$ <i>$M$ dice 0 se i due blocchi sono uguali (visto che due cose uguali verranno cifrate allo stesso modo)</i>

$Pr(PrivK^{eav}_{M, π}=1) = 1$

## CBC-mode:

Cipher Block Chaining

##### Encryption:

<div>
<img src="images/cbc_mode_enc.png" width="490"/>
<p>(estendibile fino a $t$)</p>
</div>

Introduco un <b>initialization vector</b> $r$ <b>random</b> (per avere il <b>non determinismo</b> e quindi la <b>CPA-security</b>:\
$r \leftarrow \{0,1\}^n$

##### Decryption:

<div>
<img src="images/cbc_mode_dec.png" width="490"/>
<p>(estendibile fino a $t$)</p>
</div>

##### Caratteristiche: 

- Per poter fare la decryption, $F_k$ deve essere invertibile
- Encryption <b>non è</b> parallelizzabile (per calcolare il blocco $i$-esimo, devo aver calcolato il blocco $(i-1)$-esimo)
- Decryption <b>non è</b> random access (perchè comunque dipende dal blocco precedente)
- La lunghezza del ciphertext è esattamente un blocco in più rispetto a quella del plaintext (perchè ci aggiungo $y_0=r$), ma ci va bene: $|E_k(X)|=|x|+n$

### Teorema: 

Se $F$ è PRF, allora CBC è <b>CPA-sicuro</b> 

### Esercizio: CBC-mode con $F_k$ non pseudo-random:

Ci aspettiamo che non sia CPA-sicuro perchè $F_k$ non è PRF\
Come PRF prendiamo $F_k(x) = x ⊕ k$ come l'OTP che sappiamo <b>non essere</b> pseudo-random

In realtà <b>non è neanche</b> EAV-sicuro

$PrivK^{eav}_{M,π}(n)$

$A: \quad k \leftarrow Gen(1^n) \quad b \leftarrow \{0,1\}$\
$M \rightarrow A: \quad X_0 = x_0 \ x_0 \quad X_1 = x_0 \ x_1 \quad con \ \ x_0 \neq x_1$
$M \rightarrow A: \quad E_k(x_b) $\
$M: \quad b_m \ = \ ?$

$b=0$: 

$y_0 = r \ \quad y_1 = (r ⊕ x_0) ⊕ k = r ⊕ x_0 ⊕ k \ \quad y_2 = (y_1 ⊕ x_0) ⊕ k = (r ⊕ x_0 ⊕ k ⊕ x_0) ⊕ k = r $

$b=1$: 

$y_0 = r \ \quad y_1 = (r ⊕ x_0) ⊕ k = r ⊕ x_0 ⊕ k \ \quad y_2 = (y_1 ⊕ x_1) ⊕ k = (r ⊕ x_0 ⊕ k ⊕ x_1) ⊕ k = r ⊕ x_0 ⊕ x_1$

$b_d \ \ = \ \ (y_2 == y_0) \ \ ? \ \ 0 \ \ : \ \ 1 \quad$ <i>sfruttando il risultato ottenuto semplificando $y_2$ nel caso $b=0$ (controlla se l'ultimo blocco del ciphertext è uguale al primo blocco del ciphertext</i>

$Pr(b_m = 0 \ | \ b=0) = 1 \quad$ <i>vince sempre nel caso $b=0$</i>\
$Pr(b_m = 1 \ | \ b=1) = 1 - Pr(b_m = 0 \ | \ b=1) = 1 - 0 = 1$

Come abbiamo calcolato $Pr(b_m = 0 \ | \ b=1) = 0$ ?\
$Pr(b_m = 0 \ | \ b=1)$ è la probabilità che $M$ sbagli, ovvero dica 0 quando in realtà $b=1$.\
$M$ dice 0 quando $y_2=y_0$, ma quando $b=1$, $y_2 = r ⊕ x_0 ⊕ x_1$. Per avere $y_2=y_0$, $y_2$ dovrebbe essere uguale a $r$ e questo è possibile solo quando $x_0=x_1$, ma nel caso $b=1$ questo è impossibile perchè $A$ sta cifrando $x_b=X_1=x_0 \ x_1 \neq x_0 \ x_0$.\
In altre parole, $\ Pr((x_0 ⊕ x_1) = 0) = 0$

$Pr(PrivK^{eav}_{M,π}(n)=1) \ = \ 1/2  \cdot (Pr(b_m = 0 \ | \ b=0) + Pr(b_m = 1 \ | \ b=1)) = 1/2 \cdot (1 + 1) = 1 \rightarrow$ CBC-OTP <b>non</b> è EAV-sicuro